In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
points = pd.read_csv('data/results.csv', usecols = (1,2,3,9,17), names =['raceId','driverId','constructorId','points','statusId'])
drivers = pd.read_csv('data/driver.csv', usecols = (0,1), names =['driverId','driverRef'])
races = pd.read_csv('data/races.csv', usecols = (0,1,4), names =['raceId','year','name'])
status = pd.read_csv('data/status.csv', names =['statusId','status'])
teams = pd.read_csv('data/constructors.csv', names =['constructorId','constructorRef','name','nationality','url'])

In [3]:
dict_drivers = dict(zip(drivers.values[:,0], drivers.values[:,1]))
dict_year = dict(zip(races.values[:,0], races.values[:,1]))
dict_team = dict(zip(teams.values[:,0], teams.values[:,1]))
dict_circuit = dict(zip(races.values[:,0], races.values[:,2]))
dict_status = dict(zip(status.values[:,0], status.values[:,1]))
dict_color = {'red_bull': '#1E41FF', 'mclaren': '#FF8700', 'renault': '#FFF500', 'ferrari': '#DC0000', 
              'toro_rosso': '#469BFF', 'force_india': '#F596C8','racing_point': '#F596C8', 'lotus_racing': 'g', 'virgin': 'lime', 
              'williams': 'gainsboro', 'mercedes': '#00D2BE', 'sauber': '#9B0000','alfa': '#9B0000', 'hrt': 'purple', 
              'lotus_f1': '#FFF500', 'marussia': 'k', 'caterham': 'g', 'manor': 'k', 'haas': '#F0D787',
              'bmw_sauber': '#9B0000', 'toyota': 'crimson', 'super_aguri': 'k', 'honda': 'lightcoral', 
              'spyker': 'gold','mf1': 'gold', 'spyker_mf1' : 'gold', 'bar': 'chartreuse', 'jordan': 'gold', 
              'minardi': 'grey','jaguar': '#0B6836','arrows': 'orangered','prost': 'b','benetton': 'springgreen', 
              'stewart': 'k','tyrrell': 'k', 'lola': 'orangered', 'ligier':'darkgreen','footwork':'indigo',
              'forti': 'yellowgreen','brawn':'k', 'pacific':'royalblue', 'simtek':'magenta',
              'larrousse':'darkgreen','team_lotus':'#FFF500','dallara':'c','fondmetal': 'b','march': 'orangered',
              'moda':'k', 'brabham':'magenta','lambo':'grey' ,'ags':'olive','coloni':'k','leyton':'aqua',
             'eurobrun': 'silver','osella':'b' , 'onyx': 'navy','life':'pink','rial': 'navy','zakspeed': 'pink'}

In [4]:
points['year'] = points['raceId'].map(dict_year)
points['driver'] = points['driverId'].map(dict_drivers)
points['team'] = points['constructorId'].map(dict_team)
points['status'] = points['statusId'].map(dict_status)
points['circuit'] = points['raceId'].map(dict_circuit)
points['color'] = points['team'].map(dict_color)

In [5]:
latest = points[(points['year'] >= 1986)]
sorted_latest = latest.sort_values(by = ['year', 'raceId'])
sorted_latest.head()

,raceId,driverId,constructorId,points,statusId,year,driver,team,status,circuit,color
9407,404,137,3,9.0,1,1986,piquet,williams,Finished,Brazilian Grand Prix,gainsboro
9408,404,102,32,6.0,1,1986,senna,team_lotus,Finished,Brazilian Grand Prix,#FFF500
9409,404,172,27,4.0,1,1986,laffite,ligier,Finished,Brazilian Grand Prix,darkgreen
9410,404,163,27,3.0,1,1986,arnoux,ligier,Finished,Brazilian Grand Prix,darkgreen
9411,404,84,25,2.0,11,1986,brundle,tyrrell,+1 Lap,Brazilian Grand Prix,k


In [6]:
def plot(Season):
    fig = plt.figure(figsize = (20,10))
    ax = plt.subplot(111)
    teams = list()
    season_points = sorted_latest[(sorted_latest['year'] == Season)]
    for driver in season_points['driver'].unique():
        tmp = season_points[season_points['driver'] == driver]
        cum_latest =  np.cumsum(tmp.values[:,3])
        team = tmp.values[0,7]
        if team in teams:
            ax.scatter(tmp.values[:,9], cum_latest, color = tmp.values[0,10],  edgecolor = 'k', s = 100)
            ax.plot(tmp.values[:,9], cum_latest, color = tmp.values[0,10])
        else:
            ax.scatter(tmp.values[:,9], cum_latest, color = tmp.values[0,10],  edgecolor = 'k', label = team, s = 100)
            ax.plot(tmp.values[:,9], cum_latest, color = tmp.values[0,10])
        teams.append(team)
        if np.max(cum_latest) > 20:
            ax.text(len(season_points['circuit'].unique())-0.7,np.max(cum_latest),f'{driver}')
    ax.legend(bbox_to_anchor=(0.5, 1.1), loc='upper center', borderaxespad=0.,ncol=10)
    ax.set_xticklabels(season_points['circuit'].unique(), rotation = 90)
    ax.set_ylabel('Points')
    plt.show()

In [7]:
sea = sorted_latest['year'].unique()

In [8]:
_ = widgets.interact(plot, Season = sea)

interactive(children=(Dropdown(description='Season', options=(1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, …

In [9]:
standings =  pd.read_csv('data/constructor_standings.csv', names =['constructorStandingsId','raceId','constructorId','points','position','positionText', 'wins'])

In [10]:
standings['year'] = standings['raceId'].map(dict_year)
standings['team'] = standings['constructorId'].map(dict_team)
standings['circuit'] = standings['raceId'].map(dict_circuit)
standings['color'] = standings['team'].map(dict_color)
standings['year'] = standings['year'].astype(int)

In [11]:
standings = standings[(standings['year'] >= 1986) & (standings['year'] <= 2020)]
sorted_standings = standings.sort_values(by = ['year', 'raceId'])
sorted_standings.head()

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins,year,team,circuit,color
4790,9684,404,3,9.0,1,1,1,1986,williams,Brazilian Grand Prix,gainsboro
4791,9685,404,32,6.0,3,3,0,1986,team_lotus,Brazilian Grand Prix,#FFF500
4792,9686,404,27,7.0,2,2,0,1986,ligier,Brazilian Grand Prix,darkgreen
4793,9687,404,25,2.0,4,4,0,1986,tyrrell,Brazilian Grand Prix,k
4794,9688,404,22,1.0,5,5,0,1986,benetton,Brazilian Grand Prix,springgreen


In [12]:
def graph(Participations, legend):
    fig = plt.figure(figsize = (20,10))
    ax = plt.subplot(111)
    for team in sorted_standings['team'].unique():
        tmp = sorted_standings[sorted_standings['team'] == team]
        years = tmp.values[:,7]
        teams = list()
        long = len(tmp['year'].unique())
        if long >= Participations:
            for year in tmp['year'].unique():
                tmp_year = tmp[tmp['year'] == year]
                last_race = np.max(tmp_year['raceId'].unique())
                last_race_data = tmp_year[tmp_year['raceId'] == last_race]
                position = last_race_data.values[:,4]
                if team in teams:
                    ax.scatter(year, position, c = last_race_data.values[0,10], edgecolor = 'k',s = 100)
                else:
                    ax.scatter(year, position, c = last_race_data.values[0,10], edgecolor = 'k', label= team, s = 100)
                teams.append(team)
    if legend == True:            
        ax.legend(bbox_to_anchor=(0.5, 1.05), loc='upper center', borderaxespad=0.,ncol=12)
    ax.invert_yaxis()
    ax.set_xticks(np.arange(1986,2020,1))
    ax.set_xticklabels(sorted_standings['year'].unique())
    ax.set_ylabel('Constructors Position')
    plt.show()

In [13]:
_ = widgets.interact(graph, Participations = np.flip(np.arange(1,33,1)), legend = False)

interactive(children=(Dropdown(description='Participations', options=(32, 31, 30, 29, 28, 27, 26, 25, 24, 23, …

In [14]:
laptimes =  pd.read_csv('data/lap_times.csv', names =['raceId','driverId','lap','position','time','milliseconds'])
laptimes['year'] = laptimes['raceId'].map(dict_year)
laptimes['circuit'] = laptimes['raceId'].map(dict_circuit)

In [15]:
sorted_laptimes = laptimes.sort_values(by = ['year', 'raceId'])
sorted_laptimes.head()

,raceId,driverId,lap,position,time,milliseconds,year,circuit
113794,224,30,1,4,1:45.188,105188,1996,Australian Grand Prix
113795,224,30,2,3,1:37.680,97680,1996,Australian Grand Prix
113796,224,30,3,3,1:35.759,95759,1996,Australian Grand Prix
113797,224,30,4,3,1:35.467,95467,1996,Australian Grand Prix
113798,224,30,5,3,1:35.254,95254,1996,Australian Grand Prix


In [16]:
def times(Circuit, deviation):
    colors = plt.cm.hsv(np.linspace(0, 1, len(sorted_laptimes['year'].unique())))
    fig = plt.figure(figsize = (20,10))
    ax = plt.subplot(111)
    circuit = sorted_laptimes[sorted_laptimes['circuit'] == Circuit]
    i = 0
    times = list()
    for year in circuit['year'].unique():
        tmp = circuit[circuit['year'] == year]
        lap = tmp.values[:,2]
        number_laps = np.max(lap)
        for driver in tmp['driverId'].unique():
            tmp2 = tmp[tmp['driverId'] == driver]
            laps = tmp2.values[:,2]
            if np.max(laps) == number_laps:
                time = tmp.values[:,5]/60000
                times.extend(time)
        ax.scatter(year, np.sum(times)/len(times), edgecolor = 'k', s = 100, color = colors[i])
        if deviation == True:
            ax.errorbar(year, np.sum(times)/len(times), yerr =np.std(times), capsize = 10, color = colors[i])
        i+=1
    ax.set_xticks(np.arange(1996,2020,1))
    ax.set_xticklabels(sorted_laptimes['year'].unique())
    plt.show()

In [17]:
_ = widgets.interact(times, Circuit = sorted_laptimes['circuit'].unique(), deviation = False)

interactive(children=(Dropdown(description='Circuit', options=('Australian Grand Prix', 'Brazilian Grand Prix'…